In [8]:
exit(0)

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install transformers -q

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 43.5 MB/s 
     |████████████████████████████████| 61 kB 409 kB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 895 kB 71.9 MB/s 


In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
!pip install --upgrade pandas==1.3.4

In [2]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [3]:
dev = xm.xla_device()
dev

device(type='xla', index=1)

In [4]:
import glob
import os
import pandas as pd

def load_tweets(base_path, columns=None):
    if columns is None:
        columns = ['id', 'time', 'retweet_count', 'content', 'player']

    tweets_batch = []
    for file in glob.glob(base_path):
        current_data = pd.read_csv(file, sep=";")
        current_data['player'] = file.split(os.path.sep)[-1].split(".csv")[0]
        tweets_batch.append(current_data)

    data = pd.concat(tweets_batch)
    data = data[columns]
    return data


In [5]:
from tqdm.notebook import trange, tqdm

In [ ]:
!rm -rf ./cardiffnlp

In [6]:
from scipy.special import softmax

In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import pickle


def normalize_score(negative, neutral, positive):
    return ((positive + neutral / 2) * 2) - 1 if positive > negative else -((negative + neutral / 2) * 2) + 1


def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


tweets = load_tweets("tweets/*.csv")

task = 'irony'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(dev)


Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [18]:
irony_map = {}

In [9]:
batch_size = 1024
batches = np.array_split(tweets, len(tweets) // batch_size)

In [ ]:
for batch in tqdm(batches):
    ids = batch['id'].values
    texts = batch['content'].map(lambda x: preprocess(x)).values

    encoded_input =  tokenizer(list(texts),return_tensors='pt',padding=True,truncation=True, max_length=128)
    data_input_ids = encoded_input['input_ids'].to(dev)
    data_attention_mask = encoded_input['attention_mask'].to(dev)

    output = model.forward(input_ids=data_input_ids,attention_mask=data_attention_mask)

    scores = softmax(output.logits.cpu().detach().numpy(), axis=1)

    for idx, score in enumerate(scores):
      irony = score[1]
      if irony > 0.9:
        irony_map[ids[idx]] = irony
        
with open('irony.pkl', 'wb') as file:
    pickle.dump(irony_map, file, pickle.HIGHEST_PROTOCOL)

  0%|          | 0/222 [00:00<?, ?it/s]